In [1]:
#importing necessary packages
from collections import defaultdict
import gzip
from Bio.Align import PairwiseAligner
import numpy as np
#import matplotlib.pyplot as plt
import Bio.Seq
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import Bio.Seq
import Bio.SeqIO
from Bio.SeqRecord import SeqRecord
import os
import sys
from pathlib import Path

In [2]:
import itertools

def hammingDist(str1, str2, fillchar = '-'):
    return sum([ch1 != ch2 for (ch1,ch2) in itertools.zip_longest(str1, str2, fillvalue = fillchar)])


In [8]:
library = pd.read_csv('be_cluster_scripts_BARCODE_COUNTS_2024_7_3/IDR_ABE_library_columns_renamed.csv')
library

,Hugo_Symbol,Chromosome,Start_Position,HGVSc,Variant_Index,Variant_Classification,Polyphen_Prediction,Base_Change,HGVSp_Short,Mut_PAM_Distance,...,t_depth,Annotation_Status,mutationInCis_Flag,num_tumors,Target_Sequence_50,scaffold_BC_add,Full_Seq,BC_pair,sensor_alt,gRNA_id
0,ABL1,9,133755997,ENST00000318560.5:c.1624A>G,2.0,Missense_Mutation,benign,A>G,p.R542G,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCATCTCAGGCACGTCAGTGGTGTCTCCGTGCTCTGCAGCTC,sg_1
1,ABL1,9,133755997,ENST00000318560.5:c.1624A>G,1.0,Missense_Mutation,benign,A>G,p.R542G,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATCTCAGGCACGTCAGTGGTGTCTCCGTGCTCTGCAGCTCTC,sg_2
2,ABL1,9,133760635,ENST00000318560.5:c.2958A>G,1.0,Silent,NaN,A>G,p.P986=,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GGTCCCCTGCCAGGGCCGAGGATGCTGACGGCAACGTGGAGG,sg_3
3,ABL2,1,179078007,ENST00000502732.1:c.2395A>G,1.0,Missense_Mutation,benign,T>C,p.M799V,-13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCTGGCAGTTCCTGGGAAGGGTCACTGCCATCCTATCCTGCT,sg_4
4,AFF1,4,88035564,ENST00000307808.6:c.1558A>G,1.0,Missense_Mutation,benign,A>G,p.T520A,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCGCAGCTGGCTGGCTGACTTTGGCCAGCCAGTTGTCCAGCT,sg_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,controlsgRNA_95,13,88376208,NaN,-,NaN,NaN,C>T,-,15.0,...,NaN,NaN,NaN,NaN,TTGAGCTATTGTACAGGGCTGGGGCCATGGGAGACAGTGTTGGTGG...,GGGGCCATGGGAGACAGTGTGTTTAAGAGCTATGCTGGAAACAGCA...,CATAGCGTACACGTCTCACACCGGGGCCATGGGAGACAGTGTGTTT...,F1-R4,CCACAGCCACCAACACTGTCTCCCATAGCCCCAGCCCTGTAC,sg_1668
1668,controlsgRNA_96,X,36797961,NaN,-,NaN,NaN,A>G,-,15.0,...,NaN,NaN,NaN,NaN,CCTGGACTGGTTGCAACCTAGACCCATGCACCTTCTTCATTGGTCT...,GACCCATGCACCTTCTTCATGTTTAAGAGCTATGCTGGAAACAGCA...,CATAGCGTACACGTCTCACACCGACCCATGCACCTTCTTCATGTTT...,F1-R4,TAGGGGAGACCAATGAAGAAGGTGCACGGGTCTAGGTTGCAA,sg_1669
1669,controlsgRNA_97,10,57998002,NaN,-,NaN,NaN,C>T,-,15.0,...,NaN,NaN,NaN,NaN,GGTGTGGCATGACACTGTAGGAAGTCCTCAAAACTAATCAGGGCAG...,GAAGTCCTCAAAACTAATCAGTTTAAGAGCTATGCTGGAAACAGCA...,CATAGCGTACACGTCTCACACCGAAGTCCTCAAAACTAATCAGTTT...,F1-R4,TTTCCCCTGCCCTGATTAGTTTTGAGAACTTCCTACAGTGTC,sg_1670
1670,controlsgRNA_98,4,81701927,NaN,-,NaN,NaN,A>G,-,15.0,...,NaN,NaN,NaN,NaN,TATACAAAATATGTAATCCTGTGAGACATTATATATTGACTGGGTA...,GTGAGACATTATATATTGACGTTTAAGAGCTATGCTGGAAACAGCA...,CATAGCGTACACGTCTCACACCGTGAGACATTATATATTGACGTTT...,F1-R4,TCTGTCTACCCAGTCAATATATAATGCCTCACAGGATTACAT,sg_1671


In [16]:
protos = list(library['Protospacer'])

mat = np.zeros((len(protos), len(protos)))

count_off_diag=0
for i, val in enumerate(protos):
    for k, val2 in enumerate(protos):
        dist = hammingDist(val, val2, fillchar = '-')
        if dist<=2:
            mat[i][k]+=1
            if i!=k:
                count_off_diag +=1
                print(i)
                print(k)


379
380
380
379
1347
1642
1642
1347


In [30]:
import Levenshtein as lv

In [32]:
lv.hamming('ATCG', 'ATGG')

1

In [33]:
protos = list(library['Protospacer'])

mat = np.zeros((len(protos), len(protos)))

count_off_diag=0
for i, val in enumerate(protos):
    for k, val2 in enumerate(protos):
        dist = lv.hamming(val, val2)
        if dist<=2:
            mat[i][k]+=1
            if i!=k:
                count_off_diag +=1
                print(i)
                print(k)

379
380
380
379
1347
1642
1642
1347


In [21]:
protos[379]==protos[380]

True

In [23]:
library.iloc[[379, 380]]

,Hugo_Symbol,Chromosome,Start_Position,HGVSc,Variant_Index,Variant_Classification,Polyphen_Prediction,Base_Change,HGVSp_Short,Mut_PAM_Distance,...,t_depth,Annotation_Status,mutationInCis_Flag,num_tumors,Target_Sequence_50,scaffold_BC_add,Full_Seq,BC_pair,sensor_alt,gRNA_id
379,DICER1,14,95562834,ENST00000343455.3:c.4423A>G,1.0,Missense_Mutation,benign,T>C,p.T1475A,-14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATTTTCCATTCATATGCAGAATCAGCGGTTGAAAAAGGAGAA,sg_380
380,DICER1,14,95562837,ENST00000343455.3:c.4420A>G,1.0,Missense_Mutation,benign,T>C,p.T1474A,-17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATTTTCCATTCATATGCAGAATCAGTGGCTGAAAAAGGAGAA,sg_381


In [20]:
protos[1642]

'GGAGTAGAGGAAGATGGTGA'

In [15]:
count_off_diag

4

In [4]:
hammingDist('ATCGA', 'ATGGA', fillchar = '-')

1